# The Mozilla Network Security Service (NSS)

Started in 1998 the Mozilla project was born out of Netscape. 

> Network Security Services (NSS) is a set of libraries designed to support cross-platform development of communications applications that support SSL, S/MIME, and other Internet security standards.

Those interested in the history of NSS, it can be found here (https://www-archive.mozilla.org/projects/security/pki/nss/history)[]

While NSS and OpenSSL can both accomplish the same task, they both provide some interesting ways to accomplish them. In some cases a distribution may leverage more of one method then the other, but as we will see in the later sections, this is all being unified into a common set of libraries. In some cases NSS may not be loaded with the base installation.

The one primary difference between the two projects is that NSS uses a database for storing the certificates. Along with providing an API method to access certificate function. It is common to find it being used by many application in the Linux ecosystem, some directly, others via the system. 

## Basic structure

Looking at the packaged version of the tools.

```
# rpm version
rpm -q nss --info
echo
rpm -q nss-tools --info
```

In [ ]:
%%bash
dpkg-query -s libnss3
echo 
dpkg-query -s libnss3-tools

The NSS system places the certificate storage and configuration into a directory, generally named nssdb. This is arbitrary, in the case of Firefox the default profile directory in `~/.mozilla/firefox/` is the database directory. 

This directory can be created anywhere on the system. For system level provision this is usally help in `/etc/` in Red Hat based systems it is located `/etc/pki/nssdb`, in debian based systems it is not always created from the base package, but creating it in the `/etc/pki/nssdb`  provides the same functionality.

For our examples here, we will create a new arbitary folder.

In [ ]:
%%bash
rm -rf nssdb
mkdir nssdb

Now we can generate a new instance of the nssdb with the `certutil` command.

In [ ]:
%%bash
certutil -d nssdb -N --empty-password

Looking inside we can see the structure created. 

_Note that our examples are using the BerkeleyDB version of the database structure, by using the `sql:` prefix to the database name (-d flag) it will provision it with SQLite._

In [ ]:
%%bash
ls -l nssdb

## Working with certificates

Now that we have a store for our certificates we can work with it. 

The NSS structure is more tightly coupled then the openssl structure. This provides many utilities rolled into single commands, but it does also make it more difficult to do the more simple inspection that we did with openssl. Much of the power in the NSS structure is that it can be used from both provision and consumption. We will dig a bit more into those parts in a later section. For now let us look at some simple use cases. 

Lets add the certificate for ohiolinux.org so we can show a couple of the same functions used by openssl in the NSS tool. 

In [ ]:
%%bash
certutil -d nssdb -A -n "ohiolinux" -t ",," -i certificates/ohiolinux.crt
certutil -d nssdb -L

Then viewing the certificate.

In [ ]:
%%bash
certutil -d nssdb -L -n "ohiolinux"

We can also see the certificate in ascii form.

In [ ]:
%%bash
certutil -d nssdb -L -n "ohiolinux" -a

Let us try to validate the certificate.

In [ ]:
%%bash
certutil -d nssdb -V -n "ohiolinux" -u C -e -b 191101120000Z

We need to have the chain cert added as well, we also have to set it to a trusted certificate by adding the "T,," with the -t (trust) flag.

In [ ]:
%%bash
certutil -d nssdb -A -n "olchain" -t "T,," -i certificates/olchain.crt

Checking it again.

In [ ]:
%%bash
certutil -d nssdb -V -n "ohiolinux" -u C -e -b 191101120000Z

NSS also has a module configuration that can be accessed using `modutil`, this will be important for using system level functions. For now we can see that the current nssdb does not have any additional modules associated to it.

In [ ]:
%%bash
modutil -dbdir nssdb -list

One important thing to note, that will come up again later, is the URI displayed in the module interface. 

## Some closing comments

In this section we introduced the NSS utilties. While this section is a bit lean on content, we will be revisiting the NSS utilities later as it integrates with other features we are going to discuss. 

Conceptually the NSS utilities can be used for a great deal more. While we will primarily be using openssl in the examples, I will try and also show the NSS utility equivilent. 